<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.04/IR_04_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!pip install pandas numpy torch transformers datasets scikit-learn tqdm

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [6]:
link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.04/sentences.csv?raw=true"
df = pd.read_csv(link)
texts = df['sentence'].values
labels = df['label'].values

In [7]:
# take a subset of the df 100 from each of the 5 classes
df = df.groupby('label').apply(lambda x: x.sample(n=500, random_state=42)).reset_index(drop=True)
texts = df['sentence'].values
labels = df['label'].values

In [8]:
# check if the df contains 500 instances
df.groupby('label').count()

,id,sentence,type
label,,,
0,500,476,500
1,500,500,500
2,500,500,500
3,500,500,500
4,500,500,500


In [9]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.15, random_state=42
)

In [10]:
# Create dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [11]:
# Create model class
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes=5):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.drop = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        output = self.drop(output[0][:, 0, :])
        return self.fc(output)

In [12]:
# Initialize tokenizer and create datasets
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [13]:
# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [14]:
# Initialize model and move to GPU
model = SentimentClassifier()
model = model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [15]:
# Training parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
n_epochs = 5

In [16]:
# Training loop
def train_model():
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc='Training'):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

In [17]:
# Evaluation loop
def evaluate_model():
    model.eval()
    total_loss = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

            _, predictions = torch.max(outputs, dim=1)

            total_loss += loss.item()
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    return total_loss / len(val_loader), accuracy

In [18]:
# Training
for epoch in range(n_epochs):
    print(f'\nEpoch {epoch + 1}/{n_epochs}')
    train_loss = train_model()
    val_loss, val_accuracy = evaluate_model()

    print(f'Training Loss: {train_loss:.4f}')
    print(f'Validation Loss: {val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy:.4f}')


Epoch 1/5


Evaluating: 100%|██████████| 24/24 [00:01<00:00, 17.58it/s]


Training Loss: 0.6837
Validation Loss: 0.1757
Validation Accuracy: 0.9600

Epoch 2/5


Evaluating: 100%|██████████| 24/24 [00:01<00:00, 16.40it/s]


Training Loss: 0.1183
Validation Loss: 0.1433
Validation Accuracy: 0.9547

Epoch 3/5


...

Training Loss: 0.0447
Validation Loss: 0.0884
Validation Accuracy: 0.9707

Epoch 4/5


Evaluating: 100%|██████████| 24/24 [00:01<00:00, 17.17it/s]


Training Loss: 0.0311
Validation Loss: 0.1461
Validation Accuracy: 0.9627

Epoch 5/5


Evaluating: 100%|██████████| 24/24 [00:01<00:00, 17.22it/s]

Training Loss: 0.0174
Validation Loss: 0.1529
Validation Accuracy: 0.9573


In [19]:
# Save the model
torch.save(model.state_dict(), 'sentiment_model_v2.pth')

In [27]:
# use the model for testing a sentence

e1 = "The Hamas terroriest have launch a rocket attack to major isreali cities"

In [28]:
import torch
from transformers import DistilBertTokenizer

# Assuming the SentimentClassifier class is already defined as provided

# Initialize the model and tokenizer
model = SentimentClassifier()
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Function to classify a sentence and get probabilities
def classify_sentence_prob(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Get the model's output
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    return probabilities

# Example usage
# sentence = "This is a sample sentence for classification."
probabilities = classify_sentence_prob(e1)

print("Probabilities for each class:", probabilities)

Probabilities for each class: tensor([[0.1049, 0.2062, 0.1647, 0.2857, 0.2384]])


In [29]:
import torch
from transformers import DistilBertTokenizer

# Assuming the SentimentClassifier class is already defined as provided

# Initialize the model and tokenizer
model = SentimentClassifier()
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Function to classify a sentence and get one-hot encoded vector
def classify_sentence_class(sentence):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Get the model's output
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Get the predicted class (index of the max probability)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    # Create a one-hot encoded vector
    one_hot_vector = torch.zeros(probabilities.size(1))
    one_hot_vector[predicted_class] = 1

    return one_hot_vector

# Example usage
# sentence = "This is a sample sentence for classification."
one_hot_vector = classify_sentence_class(e1)

print("One-hot encoded vector for the predicted class:", one_hot_vector)

One-hot encoded vector for the predicted class: tensor([1., 0., 0., 0., 0.])


In [30]:
def one_hot_to_class(one_hot_vector, map_class):
    # Find the index of the maximum value in the one-hot encoded vector
    class_index = torch.argmax(one_hot_vector).item()

    # Get the string representation of the class from the map_class dictionary
    class_string = map_class[class_index]

    return class_string

map_class = {0: 'pro-israeli', 1: 'pro-palestinan', 2: 'neutral', 3: 'anti-isreali', 4: 'anti-palestinian'}

one_hot_to_class(one_hot_vector, map_class)

'pro-israeli'